In [2]:
%pip install pandas
import pandas as pd
data = pd.read_csv("India Agriculture Crop Production.csv")

  Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached numpy-2.2.4-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl (11.4 MB)
Using cached numpy-2.2.4-cp312-cp312-macosx_14_0_arm64.whl (5.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
data.isnull().sum()  # Check for missing values
data = data.dropna()  # Drop rows with missing values
print(data.head())

                         State                  District      Crop     Year  \
0  Andaman and Nicobar Islands                  NICOBARS  Arecanut  2001-02   
1  Andaman and Nicobar Islands                  NICOBARS  Arecanut  2002-03   
2  Andaman and Nicobar Islands                  NICOBARS  Arecanut  2003-04   
3  Andaman and Nicobar Islands  NORTH AND MIDDLE ANDAMAN  Arecanut  2001-02   
4  Andaman and Nicobar Islands            SOUTH ANDAMANS  Arecanut  2002-03   

       Season    Area Area Units  Production Production Units     Yield  
0      Kharif  1254.0    Hectare      2061.0           Tonnes  1.643541  
1  Whole Year  1258.0    Hectare      2083.0           Tonnes  1.655803  
2  Whole Year  1261.0    Hectare      1525.0           Tonnes  1.209358  
3      Kharif  3100.0    Hectare      5239.0           Tonnes  1.690000  
4  Whole Year  3105.0    Hectare      5267.0           Tonnes  1.696296  


In [7]:
%pip install scikit-learn

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['State'] = le.fit_transform(data['State'])
data['District'] = le.fit_transform(data['District'])
data['Season'] = le.fit_transform(data['Season'])
data['Crop'] = le.fit_transform(data['Crop'])

  Using cached scikit_learn-1.6.1-cp312-cp312-macosx_12_0_arm64.whl.metadata (31 kB)
  Using cached scipy-1.15.2-cp312-cp312-macosx_14_0_arm64.whl.metadata (61 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp312-cp312-macosx_12_0_arm64.whl (11.2 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.15.2-cp312-cp312-macosx_14_0_arm64.whl (22.4 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
X = data[['State', 'District', 'Season', 'Area']]
y = data['Crop']

In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X['Area'] = scaler.fit_transform(X[['Area']])

/var/folders/ty/wzy1hvwd7bd4plh9mwvg9yyc0000gn/T/ipykernel_30464/4108444409.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Area'] = scaler.fit_transform(X[['Area']])


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [12]:
from sklearn.metrics import classification_report, accuracy_score
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.43875857409338603
              precision    recall  f1-score   support

           0       0.62      0.67      0.65       430
           1       0.37      0.38      0.38      2108
           2       0.39      0.40      0.40      1529
           3       0.50      0.49      0.49       896
           4       0.49      0.50      0.50      1153
           5       0.48      0.53      0.51       244
           6       0.65      0.65      0.65        88
           7       0.51      0.52      0.51       322
           8       0.29      0.28      0.29       909
           9       0.70      0.71      0.70       559
          10       0.35      0.36      0.36       984
          11       0.39      0.39      0.39      1216
          12       0.20      0.19      0.19       351
          13       1.00      1.00      1.00         1
          14       0.37      0.37      0.37      1734
          15       0.36      0.39      0.37       997
          16       0.40      0.40      0.40       9

In [16]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('model', RandomForestClassifier(random_state=42))
])

# Define the parameter distribution
param_dist = {
    'model__n_estimators': randint(50, 200),
    'model__max_depth': [None, 10, 20],
    'model__min_samples_split': randint(2, 10)
}

# Define RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=pipeline, 
    param_distributions=param_dist, 
    n_iter=10,  # Number of parameter settings sampled
    cv=3,       # 3-fold cross-validation
    scoring='accuracy', 
    random_state=42,
    n_jobs=-1   # Use all available CPU cores
)

# Fit the RandomizedSearchCV
random_search.fit(X_train, y_train)

# Print the best parameters
print("Best Parameters:", random_search.best_params_)

/Users/vvmohith/Desktop/OOAD-proj/.venv/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


Best Parameters: {'model__max_depth': None, 'model__min_samples_split': 6, 'model__n_estimators': 152}


In [17]:
best_model = random_search.best_estimator_

# Evaluate on the test set
from sklearn.metrics import classification_report, accuracy_score

y_pred = best_model.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Test Accuracy: 0.45758853164519775
Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.69      0.65       430
           1       0.39      0.41      0.40      2108
           2       0.41      0.42      0.41      1529
           3       0.51      0.51      0.51       896
           4       0.51      0.53      0.52      1153
           5       0.52      0.54      0.53       244
           6       0.62      0.64      0.63        88
           7       0.51      0.53      0.52       322
           8       0.31      0.31      0.31       909
           9       0.68      0.75      0.71       559
          10       0.36      0.37      0.36       984
          11       0.41      0.40      0.40      1216
          12       0.21      0.18      0.20       351
          13       0.00      0.00      0.00         1
          14       0.38      0.39      0.38      1734
          15       0.37      0.40      0.39       997
          16       0.4

/Users/vvmohith/Desktop/OOAD-proj/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/vvmohith/Desktop/OOAD-proj/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/vvmohith/Desktop/OOAD-proj/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{m

In [18]:
import joblib

# Save the model
joblib.dump(best_model, "crop_recommendation_model.pkl")
print("Model saved as crop_recommendation_model.pkl")

Model saved as crop_recommendation_model.pkl


In [27]:
import pandas as pd

# Define column names based on your training data
feature_names = ['State', 'District', 'Season', 'Area']
sample_input = [[4, 1, 0, 5.0]]  # State, District, Season, Area (Hectare)
sample_input_df = pd.DataFrame(sample_input, columns=feature_names)

# Predict using the pipeline
sample_prediction = best_model.predict(sample_input_df)

# Decode the crop label back to the crop name
crop_name = le.inverse_transform(sample_prediction)

# Print the recommended crop
print(f"Recommended Crop: {crop_name[0]}")

Recommended Crop: Rice
